## Unzip The Raw Data

In [1]:
!./unzip.sh UCI_HAR_Dataset.zip 2>&1 > /dev/null
!pip install -q --no-dependencies tflite-model-maker
!pip install -q tensorflow==2.11.0
!pip show tensorflow

Name: tensorflow
Version: 2.11.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/erklarungsnot/miniconda3/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: keras-tuner, tflite-model-maker


In [2]:
import os, glob
import warnings

import numpy as np

import tensorflow as tf
# tf.enable_control_flow_v2() # only for tf 1.0

import tensorflow.keras as keras
from tensorflow.keras.layers import LSTM, BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L1L2

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

warnings.filterwarnings("ignore")
warnings.filterwarnings(
    "ignore", message="calling BaseResourceVariable.__init__.*constraint is deprecated"
)

2023-05-08 18:30:29.523567: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 18:30:29.765605: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-08 18:30:29.765628: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-08 18:30:30.763659: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Building The Dataset

In [3]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape) + [nb_classes])


def load_y(subset):
    # Get the path
    path = f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/y_{subset}.txt"

    # Read the file
    y = np.loadtxt(path, delimiter=",", dtype=int)

    # # One-hot encode labels
    one_hot_labels = get_one_hot(y - 1, len(np.unique(y)))
    if subset == "train":
        assert one_hot_labels.shape == (
            7352,
            6,
        ), f"Wrong dimensions: {one_hot_labels.shape} should be (7352, 6)"
    if subset == "test":
        assert one_hot_labels.shape == (
            2947,
            6,
        ), f"Wrong dimensions: {one_hot_labels.shape} should be (2947, 6)"
    assert (
        y[0] - 1 == np.where(one_hot_labels[0] == np.max(one_hot_labels[0]))[0][0]
    ), f"Value mismatch {np.max(one_hot_labels[0])[0][0]} vs {y[13] - 1}"
    return one_hot_labels


def build_data(subset):
    if subset not in ["train", "val", "test"]:
        raise Exception(f"Invalid subset: {subset}")

    folder_path = f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/Inertial Signals/"

    # Get all signal files in folder
    signal_files = glob.glob(os.path.join(folder_path, "*.txt"))
    # print(signal_files)

    assert len(signal_files) == 9, f"No signal files found in {folder_path}"
    np.loadtxt(signal_files[0]).shape
    # print(f"{signal_shape}")

    # Determine signal order based on file names
    signal_order = [
        "body_acc_x_",
        "body_acc_y_",
        "body_acc_z_",
        "body_gyro_x_",
        "body_gyro_y_",
        "body_gyro_z_",
        "total_acc_x_",
        "total_acc_y_",
        "total_acc_z_",
    ]

    # file_prefix = "UCI_HAR_Dataset/UCI_HAR_Dataset/train/Inertial Signals/"
    # file_suffix = ".txt"
    signal_files = [
        f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/Inertial Signals/{x}{subset}.txt"
        for x in signal_order
    ]

    # Load signal data from each file and append to signals_data list
    signals_data = [np.loadtxt(x) for x in signal_files]

    # Transpose signal data array so that shape is (number of samples, number of timesteps, number of signals)
    signals_data = np.transpose(signals_data, (1, 2, 0))

    # Verify final shape of combined data
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    if subset == "train":
        assert signals_data.shape == (7352, 128, len(signal_files))
    else:
        assert signals_data.shape == (2947, 128, len(signal_files))
    return signals_data


def load_data():
    return build_data("train"), load_y("train"), build_data("test"), load_y("test")


# Loading the train and test data
X_train, y_train, X_test, y_test = load_data()


In [4]:
first_sample = X_train[0]
first_timestep = first_sample[0]
assert len(first_sample) == 128
assert first_timestep[0] == 1.8085150e-004, print(first_timestep[0])
assert first_timestep[1] == 1.0766810e-002, print(first_timestep[1])


In [5]:
# Debug
assert X_train.shape == (7352, 128, 9), print(
    "Expected shape: (7352, 128, 9) get", X_train.shape
)
assert X_test.shape == (2947, 128, 9), print(
    "Expected: (2947, 128, 9) get", X_test.shape
)
assert y_train.shape == (7352, 6), print("Expected: (7352, 6) get", y_train.shape)
assert y_test.shape == (2947, 6), print("Expected: (2947, 6) get", y_test.shape)
assert len(X_train[0][0]) == 9, print("Signals numbers not match")


In [6]:
import os
import pickle

# Create the "assets" folder if it does not exist
if not os.path.exists("assets"):
    os.mkdir("assets")

# Create the "assets/data" folder if it does not exist
data_folder = os.path.join("assets", "data")
if not os.path.exists(data_folder):
    os.mkdir(data_folder)


def save_data_to_pickle_shards(data, data_name, data_folder):
    # Check if the data already exists
    filename = os.path.join(data_folder, f"{data_name}_0.pickle")
    if os.path.exists(filename):
        print(f"{data_name} already exists in {data_folder}. Skipping data saving.")
        return

    if not os.path.exists(os.path.join(data_folder)):
        os.makedirs(os.path.join(data_folder))

    # Serialize your data
    serialized_data = pickle.dumps(data)

    # Split the serialized data into smaller chunks
    chunk_size = 50 * 1024 * 1024  # 50 megabytes
    chunks = [
        serialized_data[i : i + chunk_size]
        for i in range(0, len(serialized_data), chunk_size)
    ]

    # Save each chunk to a file in the "asset/data" folder
    for i, chunk in enumerate(chunks):
        filename = os.path.join(data_folder, f"{data_name}_{i}.pickle")
        with open(filename, "wb") as f:
            f.write(chunk)


In [7]:
save_data_to_pickle_shards(X_train, "X_train", data_folder)
save_data_to_pickle_shards(y_train, "y_train", data_folder)
save_data_to_pickle_shards(X_test, "X_test", data_folder)
save_data_to_pickle_shards(y_test, "y_test", data_folder)


X_train already exists in assets/data. Skipping data saving.
y_train already exists in assets/data. Skipping data saving.
X_test already exists in assets/data. Skipping data saving.
y_test already exists in assets/data. Skipping data saving.


In [8]:
import os
import pickle


def load_data_from_pickle_shards(data_name, data_folder):
    # Find all pickle files that match the data name
    files = sorted(
        [
            os.path.join(data_folder, f)
            for f in os.listdir(data_folder)
            if f.startswith(data_name)
        ]
    )

    # Load the data from each file
    data = b""
    for filename in files:
        with open(filename, "rb") as f:
            data += f.read()

    # Deserialize the data
    return pickle.loads(data)


# Load the data from the pickle shards
loaded_X_train = load_data_from_pickle_shards("X_train", data_folder)
loaded_y_train = load_data_from_pickle_shards("y_train", data_folder)
loaded_X_test = load_data_from_pickle_shards("X_test", data_folder)
loaded_y_test = load_data_from_pickle_shards("y_test", data_folder)


In [9]:
# Check if the loaded data matches the original data
assert loaded_X_train.shape == X_train.shape
assert loaded_y_train.shape == y_train.shape
assert loaded_X_test.shape == X_test.shape
assert loaded_y_test.shape == y_test.shape

assert (loaded_X_train == X_train).all()
assert (loaded_y_train == y_train).all()
assert (loaded_X_test == X_test).all()
assert (loaded_y_test == y_test).all()


In [10]:
# function to count the number of classes
def count_classes(y):
    return len(set([tuple(category) for category in y]))


timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = count_classes(y_train)

# Initializing parameters
n_epochs = 30
n_batch = 16

# Bias regularizer value - we will use elasticnet
regularizer = L1L2(0.01, 0.01)

print(f"Timesteps: {timesteps}")
print(f"Input dimention: {input_dim}")
print(f"Total samples: {len(X_train)}")


Timesteps: 128
Input dimention: 9
Total samples: 7352


## Training

In [11]:
# Model execution
model = Sequential([
    LSTM(
        units=64,
        input_shape=(timesteps, input_dim),
        return_sequences=True,
        bias_regularizer=regularizer,
    ),
    BatchNormalization(),
    Dropout(0.50),
    LSTM(units=48),
    Dropout(0.50),
    Dense(units=n_classes, activation="sigmoid"),
])
model.summary()


2023-05-08 18:30:34.793912: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-08 18:30:34.793987: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-08 18:30:34.794022: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-08 18:30:34.794076: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-05-08 18:30:34.921342: W tensorfl

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128, 64)           18944     
                                                                 
 batch_normalization (BatchN  (None, 128, 64)          256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 48)                21696     
                                                                 
 dropout_1 (Dropout)         (None, 48)                0         
                                                                 
 dense (Dense)               (None, 6)                 294       
                                                        

In [12]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# Training the model
model.fit(
    X_train,
    y_train,
    batch_size=n_batch,
    validation_data=(X_test, y_test),
    epochs=n_epochs,
)
model.save("assets/model-backup.h5")


Epoch 1/30
460/460 [==============================] - 25s 50ms/step - loss: 1.6676 - accuracy: 0.7199 - val_loss: 1.8056 - val_accuracy: 0.5806
Epoch 2/30
460/460 [==============================] - 22s 48ms/step - loss: 0.7064 - accuracy: 0.8999 - val_loss: 0.5148 - val_accuracy: 0.8955
Epoch 3/30
460/460 [==============================] - 22s 49ms/step - loss: 0.3176 - accuracy: 0.9271 - val_loss: 0.2740 - val_accuracy: 0.9091
Epoch 4/30
460/460 [==============================] - 23s 49ms/step - loss: 0.2100 - accuracy: 0.9283 - val_loss: 0.2886 - val_accuracy: 0.9023
Epoch 5/30
460/460 [==============================] - 23s 49ms/step - loss: 0.2059 - accuracy: 0.9302 - val_loss: 0.2129 - val_accuracy: 0.9209
Epoch 6/30
460/460 [==============================] - 23s 50ms/step - loss: 0.1988 - accuracy: 0.9305 - val_loss: 0.9239 - val_accuracy: 0.6871
Epoch 7/30
460/460 [==============================] - 23s 49ms/step - loss: 0.1749 - accuracy: 0.9351 - val_loss: 0.3477 - val_accuracy:

## Verifying Rebuilt Database

In [19]:
import numpy as np

success_thresh = 10
passed = 0

def compare_array_halves(array1, array2):
    n = len(array1) // 2
    return np.array_equal(array1[:n], array2[n:])


for i in range(len(loaded_X_train)):
    for j in range(len(loaded_X_train)):
        if i == j:
            continue
        result = compare_array_halves(loaded_X_train[i], loaded_X_train[j])
        if result:
            passed += 1
            print(
                f"The first half of array{i} is equal to the second half of array{j}."
            )
            break
    if passed == success_thresh:
        print(f"Success threshhold passed, stopping check")
        break


The first half of array1 is equal to the second half of array0.
The first half of array2 is equal to the second half of array1.
The first half of array3 is equal to the second half of array2.
The first half of array4 is equal to the second half of array3.
The first half of array5 is equal to the second half of array4.
The first half of array6 is equal to the second half of array5.
The first half of array7 is equal to the second half of array6.
The first half of array8 is equal to the second half of array7.
The first half of array9 is equal to the second half of array8.
The first half of array10 is equal to the second half of array9.
Success threshhold passed, stopping check


## Rebuilding Model From Backup

In [20]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("assets/model-backup.h5")


### Checking The Rebuilt Model From Backup File

In [21]:
test_input = X_test[0].reshape((1, 128, 9))

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)


1/1 [==============================] - 0s 415ms/step


In [22]:
assert np.array(model.predict(test_input)).argmax() + 1 == np.array(y_test[0]).argmax() + 1


1/1 [==============================] - 0s 22ms/step


In [23]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("assets/model-backup.h5")


## Converting Keras Model to TFLite Model

In [24]:
import tensorflow as tf

model_path = 'assets/model-backup.h5'
model = tf.keras.models.load_model(model_path)

model_path_alt = 'assets/saved/'
tf.saved_model.save(model, model_path_alt)

tf.keras.backend.clear_session()

converter = tf.lite.TFLiteConverter.from_saved_model(model_path_alt)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()

with open('assets/model_lstm2.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: assets/saved/assets


INFO:tensorflow:Assets written to: assets/saved/assets
2023-05-08 18:43:10.720295: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-08 18:43:10.720324: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-08 18:43:10.720437: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: assets/saved/
2023-05-08 18:43:10.730569: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-08 18:43:10.730630: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: assets/saved/
2023-05-08 18:43:11.325478: W tensorflow/compiler/mlir/lite/flatbuffer_export.cc:2046] TFLite interpreter needs to link Flex delegate in order to run the model since it contains the following Select TFop(s):
Flex ops: FlexTensorListReserve, FlexTensorListSetItem, FlexTensorListStack
Details:
	tf.TensorListReserve(tensor<2xi32>, tenso